In [ ]:
import cv2
import matplotlib.pyplot as plt

from penalty_vision.detection import PlayerDetector, PoseDetection
from penalty_vision.utils import Config
from penalty_vision.utils.frame_utils import resize_frame
from penalty_vision.utils.ioutils import  choice_random_image

config_path = "<Path>"
config = Config(config_path)

frame_path = choice_random_image(config.frame_dir)
img = cv2.imread(frame_path)
img = resize_frame(frame=img, target_size=(1420, 780))

player_detector = PlayerDetector(model_name=config.checkpoint_path)
player_detections = player_detector.detect_kicker(frame=img)
frame = player_detector.draw_kicker(img, detections=player_detections)

pose_detection = PoseDetection()
landmarks = pose_detection.extract_pose_landmarks(frame=img, bbox=player_detections[0]['bbox'])
frame = pose_detection.draw_pose(frame, landmarks)
pose_detection.release()

frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 6))
plt.imshow(frame_rgb)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np

from penalty_vision.modules.player_tracking import PlayerTracker
from penalty_vision.processor.video_processor import VideoProcessor
from penalty_vision.utils import Config
from penalty_vision.utils.ioutils import choice_random_video
from penalty_vision.detection import PlayerDetector


config_path = r"<Path>"
config = Config(config_path)
video_path = choice_random_video(video_dir=config.video_dir)
frames = VideoProcessor(video_path).extract_all_frames_as_array()

player_detector = PlayerDetector(model_name=config.checkpoint_path, tracker=config.tracker_config)
player_tracker = PlayerTracker(player_detector)
detections = player_tracker.track_frames(frames)
print(f"Detected: {len(detections)}")